In [4]:
#Importing all these tools.  I usually need one or the other.
#For now lets just grab them all.  Some might not be used.
from math import sqrt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pydataset import data
import statistics
import acquire
import prepare
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import explore
import warnings
warnings.filterwarnings("ignore")

### Given the following confusion matrix, evaluate (by hand) the model's performance.

In [2]:
data = {'actual_dog':['46','7'],'actual_cat':['13','34']}
df = pd.DataFrame(data, columns = ['actual_dog','actual_cat'], index=['pred_dog','pred_cat'])
df

,actual_dog,actual_cat
pred_dog,46,13
pred_cat,7,34


#### In the context of this problem, what is a false positive?
    False positive would be Not a Dog but predicted a Dog
#### In the context of this problem, what is a false negative?
    False negative is that IS a Dog but predicted a Cat
#### How would you describe this model?
    Crosstab of predicting dogs and cats

### You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

Use the predictions dataset and pandas to help answer the following questions:

### An internal team wants to investigate the cause of the manufacturing defects. 

In [7]:
df = pd.read_csv("c3.csv")

In [42]:
df.head(3)

,actual,model1,model2,model3,baseline_prediction
0,No Defect,No Defect,Defect,No Defect,No Defect
1,No Defect,No Defect,Defect,Defect,No Defect
2,No Defect,No Defect,Defect,No Defect,No Defect


In [28]:
df.actual.value_counts()

No Defect    184
Defect        16
Name: actual, dtype: int64

In [29]:
pd.crosstab(df.model1, df.actual)

actual,Defect,No Defect
model1,,
Defect,8,2
No Defect,8,182


Postive = Defects<br>
Negative = No Defects

TP = Actual has Defects and Model has Defects<br>
FP = Actual has no Defects but Model has Defects<br>
FN = Actual has Defects but Model has no Defects<br>
TN = Actual has no Defects and Model has no Defects

### They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [43]:
df['baseline_prediction'] = 'No Defect' #Most frequent label in actual
df.head(3)

,actual,model1,model2,model3,baseline_prediction
0,No Defect,No Defect,Defect,No Defect,No Defect
1,No Defect,No Defect,Defect,Defect,No Defect
2,No Defect,No Defect,Defect,No Defect,No Defect


In [44]:
# Recall - calculate on subset of data
# only look at subset of actual positives (i.e actual == coffee)
subset = df[df.actual == 'Defect'] #postive 
subset.head(3)

,actual,model1,model2,model3,baseline_prediction
13,Defect,No Defect,Defect,Defect,No Defect
30,Defect,Defect,No Defect,Defect,No Defect
65,Defect,Defect,Defect,Defect,No Defect


In [50]:
model_accuracy = (df.actual == df.model1).mean()
print(f'   model1 accuracy: {model_accuracy:.2%}')

model_accuracy = (df.actual == df.model2).mean()
print(f'   model2 accuracy: {model_accuracy:.2%}')

model_accuracy = (df.actual == df.model3).mean()
print(f'   model3 accuracy: {model_accuracy:.2%}\n')

baseline_accuracy = (df.actual == df.baseline_prediction).mean()
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model1 accuracy: 95.00%
   model2 accuracy: 56.00%
   model3 accuracy: 55.50%

baseline accuracy: 92.00%


### Which evaluation metric would be appropriate here? 
    I would focus on overall Accuracy for the evaluation metric.

### Which model would be the best fit for this use case?
    I would pick the model with the most defects or the worst accuracy.  In our duck case, I would pick Model 3 @ 55% Accuracy

### Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect.
    My first thought is how do I get my hands on one of these defective ducks

In [58]:
model_recall = (subset.actual == subset.model1).mean()
print(f'   model1 recall: {model_recall:.2%}')

model_recall2 = (subset.actual == subset.model2).mean()
print(f'   model2 recall: {model_recall2:.2%}')

model_recall3 = (subset.actual == subset.model3).mean()
print(f'   model3 recall: {model_recall3:.2%}')

   model1 recall: 50.00%
   model2 recall: 56.25%
   model3 recall: 81.25%


### Which evaluation metric would be appropriate here?
    I would use recall to get only the ones that are defects.
### Which model would be the best fit for this use case?
    I would use Model 3. It had the highest recall percentage